# Load libraries and paths

In [1]:
import os
import subprocess
import numpy as np
import pandas as pd
import geopandas as gpd
import openpyxl
import matplotlib.pyplot as plt
import sys

In [2]:
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from variables import gdal_path, ancillary_POPdata_folder_path, ancillary_data_folder_path, city, createFolder, unzip
path_census = ancillary_POPdata_folder_path + '/rawData/CENSUS/'
censusPathAdm = ancillary_data_folder_path + "/adm/CensusTracts/"
censusMunipifolder = ancillary_POPdata_folder_path + '/CensusTractsMunicipality/'
createFolder(censusMunipifolder)
censusProvifolder = ancillary_POPdata_folder_path + '/CensusTractsProvince/'
createFolder(censusProvifolder)

------------------------------ Folder already exists------------------------------
------------------------------ Folder already exists------------------------------


# Get administrative boundaries

In [ ]:
#The rawboundariesFolder contains the administrative boundaries downlowaded by https://www.istat.it/it/archivio/222527 for 1991, 2001, 2011
rawboundariesFolder = ancillary_data_folder_path + '/adm/raw_boundaries/'
boundariesFolder = ancillary_data_folder_path + '/adm/boundaries/'

In [ ]:
for i in os.listdir(boundariesFolder):
    year = i.split('Limiti')[1].split('_')[0]
    folderName = '{}_adm_bound'.format(year)
    unzip(boundariesFolder + i, rawboundariesFolder, folderName)

In [ ]:
def SelectProvince(gdal_path,csPath, rawboundariesFolder, ancillary_data_folder_path, year,outname): 
    cmd_tif_merge = "{0}/ogr2ogr.exe -s_srs EPSG:32632 \
                    -t_srs EPSG:3035 -where 'COD_PROV=58' -f GPKG {1}/{2}_{4}.gpkg \
                    {3}".format(gdal_path, boundariesFolder, year, csPath, outname)
    print(cmd_tif_merge)
    subprocess.call(cmd_tif_merge, shell=False)

In [ ]:
year_list= [1991, 2001, 2011]
for year in year_list:
    # Select the province of Roma
    csPath = os.path.dirname(rawboundariesFolder) + '/{0}_adm_bound/Limiti{0}_g/Prov{0}_g/Prov{0}_g_WGS84.shp'.format(year)
    outname = 'prov_roma'
    SelectProvince(gdal_path,csPath, rawboundariesFolder, ancillary_data_folder_path, year,outname)
    #Select the municipalities in the province of Roma
    csPath1 = os.path.dirname(rawboundariesFolder) + '/{0}_adm_bound/Limiti{0}_g/Com{0}_g/Com{0}_g_WGS84.shp'.format(year)
    outname1 = 'com_roma'
    SelectProvince(gdal_path, csPath1, rawboundariesFolder, ancillary_data_folder_path, year,outname1)

# Census Data
### This part joins and cleans the data for the total population, age groups and migrant groups

## EDITING DATA FOR CENSUS TRACTS 1991, 2001, 2011

In [7]:
print("------------------------------ EDITING DATA FOR CENSUS TRACTS 1991, 2001, 2011 ------------------------------")

xls = pd.ExcelFile(path_census + 'Data_census_tracts1991_2001_2011.xlsx')
# Now you can list all sheets in the file
xls.sheet_names
for i in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=i, header=0)
    year = i
    with open(path_census + '{}_summary.txt'.format(i),'w') as out:
        df.info(buf = out)
    
    gdf = gpd.read_file(censusPathAdm + "{}_CensusTracts.geojson".format(year))
    print(gdf.crs)
    selectedgeoColumns = gdf[[ "COD_ISTAT",  "PRO_COM",  "SEZ{}".format(year) , "SEZ" ,  "Shape_Leng" , "Shape_Area" , "geometry" ]]
    geo_frame = selectedgeoColumns.copy()
    ndf = df.set_index('SEZ{}'.format(year)).join(geo_frame.set_index('SEZ{}'.format(year)),lsuffix ='_l')
    for r in ndf.columns:
        if r.endswith('_l'):
            ndf = ndf.drop(columns=['{}'.format(r)])
    new_df = gpd.GeoDataFrame(ndf,geometry='geometry')
    print(ndf.head(2))
    
    new_df.to_file(censusProvifolder + '{0}_provCensusTracts.geojson'.format(year),driver='GeoJSON',crs='EPSG:3035')

    #Make the selection only for the municipality of Rome
    if 'COMUNE' in new_df.columns:
        municipal_df = new_df.loc[new_df['COMUNE'] == 'Roma']
        print(municipal_df.head())
    elif 'COD_COM' in new_df.columns:
        municipal_df = new_df.loc[new_df['COD_COM'] == 91]
        print(municipal_df.head())
    else:
        print('nothing')
    municipal_df.to_file(censusMunipifolder + '{0}_komCensusTracts.geojson'.format(i),driver='GeoJSON',crs='EPSG:3035')    

------------------------------ EDITING DATA FOR CENSUS TRACTS 1991, 2001, 2011 ------------------------------
epsg:3035
              COD_PRO  COD_COM  SEZIONE  ISOLATO  Total population  \
SEZ1991                                                              
580010000001       58        1        1        0               314   
580010000002       58        1        2        0               452   

              Population 0-19  Population 20-29  Population 30-44  \
SEZ1991                                                             
580010000001               71                46                69   
580010000002              142                63               104   

              Population 45-64  Population 65+  ...  Dwellings - non occupied  \
SEZ1991                                         ...                             
580010000001                55              73  ...                       118   
580010000002                75              68  ...                        78  

In [8]:
year_listA =[1991,2001,2011]
for year in year_listA:
    for file in os.listdir(censusMunipifolder):
        if file.startswith('{}_'.format(year)):
            
            fileName = file.split('.')[0]
            df = gpd.read_file(censusMunipifolder + file)
            df.rename(columns={'Total population': 'l1_totalpop', 'Population with foreign citizenship and stateless': 'totalMig', 'Population with foreign citizenship EU&EFTA': 'EuropeEUnoLocal', 
            'Population with foreign citizenship Non EU&EFTA': 'EuropeNotEU', 'Population with foreign citizenship - Europe': 'EuropenoLocal','Population with foreign citizenship - Africa': 'Africa',
            'Population with foreign citizenship - America': 'America','Population with foreign citizenship - Asia': 'Asia', 'Population with foreign citizenship - Oceania': 'Oceania',
            'Population with foreign citizenship -stateless': 'sta'}, inplace=True)
            df['ita'] = df['l1_totalpop'] - df['totalMig']
            df['Asia & Oceania'] = df['Asia'] + df['Oceania']
            #Remove the standard columns from the unique Attributes and write file
            select = ['totalMig', 'ita', 'EuropeEUnoLocal', 'EuropenoLocal','EuropeNotEU','Africa','America', 'Asia','Oceania','Asia & Oceania', 'sta']

            for j in select:
                try:
                    # Calculate percentage of each migrant group per total population
                    df['Z0_{}'.format(j)] = (df['{}'.format(j)]/df['l1_totalpop'])*100 #totalPop
                except ZeroDivisionError:
                    df['Z0_{}'.format(j)] = 0
                try:
                    # Calculate percentage of each migrant group per total migration
                    df['Z1_{}'.format(j)] = (df['{}'.format(j)]/df['totalMig'])*100 #totalMig
                except ZeroDivisionError:
                    df['Z1_{}'.format(j)] = 0
            # Calculate percentage of each migrant group per total population
            df['Z0_ita'] = (df['ita']/df['l1_totalpop'])*100 #totalPop
            df.drop('Z1_totalMig', axis=1, inplace=True)
            df.to_file(censusMunipifolder + '{0}Divs.geojson'.format(fileName),driver='GeoJSON',crs='EPSG:3035')

## EDITING DATA FOR CENSUS TRACTS 2016-2019

In [3]:
# Load spatial data for census tracts
gdf11 = gpd.read_file(censusPathAdm + "2011_CensusTracts.geojson")
print(gdf11.crs)
selectedgeoColumns = gdf11[[ "COD_REG",  "COD_ISTAT",  "PRO_COM",  "SEZ2011" , "SEZ" ,  "Shape_Leng" , "Shape_Area" , "geometry" ]]
geo_frame11 = selectedgeoColumns.copy()

epsg:3035


In [4]:
print("------------------------------ EDITING DATA FOR CENSUS TRACTS 2016-2019 ------------------------------")
xls = pd.ExcelFile(path_census + 'ROME - Population size and structure 2016-19.xlsx')
# Now you can list all sheets in the file
print(xls.sheet_names)
for i in xls.sheet_names:
    year = i.split('.')[-1]
    df = pd.read_excel(xls, sheet_name=i, header=0)
    with open(path_census + '{}_summary.txt'.format(year),'w') as out:
        df.info(buf = out)
    ndf = df.set_index('Census tracts_new').join(geo_frame11.set_index('SEZ2011'))
    new_df = gpd.GeoDataFrame(ndf,geometry='geometry')
    print(new_df.head(3))
    new_df.to_file(censusMunipifolder + '{0}_komCensusTracts.geojson'.format(year),driver='GeoJSON',crs='EPSG:3035')

------------------------------ EDITING DATA FOR CENSUS TRACTS 2016-2019 ------------------------------
['ROME 1.1.2016', 'ROME 1.1.2017', 'ROME 1.1.2018', 'ROME 1.1.2019']
                   Census tracts (sezioni di censimento)  Total population   \
Census tracts_new                                                             
580911010001                                     1010001               83.0   
580911010002                                     1010002               80.0   
580911010003                                     1010003               19.0   

                   Foreigners  Italians  Africa  America  Asia & Oceania  \
Census tracts_new                                                          
580911010001             26.0      57.0     0.0      7.0             0.0   
580911010002             10.0      70.0     0.0      0.0             5.0   
580911010003              5.0      14.0     0.0      0.0             0.0   

                     EU  Europe (non EU)  Italians 

## EDITING DATA FOR CENSUS TRACTS 2015, 2020

In [5]:
year_list =[2015, 2020]
for year in year_list:
    xls = pd.ExcelFile(path_census + 'ROME - Population size and structure {}.xlsx'.format(year))
    df = pd.read_excel(xls, sheet_name=0, header=0)
    df = df.iloc[:-1 , :]
    with open(path_census + '{}_summary.txt'.format(year),'w') as out:
        df.info(buf = out)
    df['Census tracts_new']= pd.to_numeric(df['Census tracts_new'])
    ndf = df.set_index('Census tracts_new').join(geo_frame11.set_index('SEZ2011'))
    new_df = gpd.GeoDataFrame(ndf, geometry='geometry')
    new_df.to_file(censusMunipifolder + '{0}_komCensusTracts.geojson'.format(year),driver='GeoJSON',crs='EPSG:3035')

## HARMONIZING DATA FOR CENSUS TRACTS 2015-2020

In [6]:
year_list =[2015, 2016, 2017, 2018, 2019, 2020] #, 2016, 2017, 2018, 2019, 2020
for year in year_list:
    for file in os.listdir(censusMunipifolder):
        if file.startswith('{}_'.format(year)):
            fileName = file.split('.')[0]
            df = gpd.read_file(censusMunipifolder + file)
            df.rename(columns={'Census tracts_new': 'SEZ2011', 'Total population ': 'l1_totalpop', 'Foreigners': 'totalMig', 'Italians': 'ita', 
            'Europe (non EU)': 'EuropeNotEU', 'EU': 'EuropeEUnoLocal'}, inplace=True)
            df['oth'] = df['totalMig'] - ( df['Asia & Oceania'] + df['Africa'] + df['America'] + df['EuropeNotEU'] + df['EuropeEUnoLocal'])
            df['EuropenoLocal'] = df['EuropeNotEU'] + df['EuropeEUnoLocal']
            #Remove the standard columns from the unique Attributes and write file
            select = ['totalMig', 'ita', 'EuropeEUnoLocal', 'EuropenoLocal','EuropeNotEU','Africa','America', 'Asia & Oceania', 'oth']

            for j in select:
                try:
                    # Calculate percentage of each migrant group per total population
                    df['Z0_{}'.format(j)] = (df['{}'.format(j)]/df['l1_totalpop'])*100 #totalPop
                except ZeroDivisionError:
                    df['Z0_{}'.format(j)] = 0
                try:
                    # Calculate percentage of each migrant group per total migration
                    df['Z1_{}'.format(j)] = (df['{}'.format(j)]/df['totalMig'])*100 #totalMig
                except ZeroDivisionError:
                    df['Z1_{}'.format(j)] = 0
            # Calculate percentage of each migrant group per total population
            df['Z0_ita'] = (df['ita']/df['l1_totalpop'])*100 #totalPop
            df.drop('Z1_totalMig', axis=1, inplace=True)
            df.to_crs('epsg:3035')
            df.to_file(censusMunipifolder + '/{0}Divs.geojson'.format(fileName),driver='GeoJSON',crs='EPSG:3035')

# Migration and Movement in Urban Zones
This part cleans and joins the data for migration and internal migration for 2009, 2011, 2015, 2016, 2017, 2018 

In [ ]:
# Migration and Movement 
# Join the spatial data of the census tracts with population for 23 fields
path_rawPopData = ancillary_POPdata_folder_path + '/rawData/'
xls = pd.ExcelFile(path_rawPopData + 'ROME - Migration and removals 2011-15-16-17-18.xlsx')
# Now you can list all sheets in the file
xls.sheet_names
for i in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=i, header=0, skiprows=1)
    print(df.head())
    censusPathGPD = ancillary_data_folder_path + '/adm/rom_urbanzones.gpkg'
    gdf = gpd.read_file(censusPathGPD).to_crs('epsg:3035')
    ndf = df.set_index('ZURB code').join(gdf.set_index('COD_Z_URB'), lsuffix='_l')
    for g in ndf.columns:
        if g.endswith('_l'):
            ndf = ndf.drop(columns=[g])
    #ndf = ndf.dropna(axis=0, subset=['Total population'])
    new_df = gpd.GeoDataFrame(ndf,geometry='geometry')
    print(ndf.head)
    outfolder= ancillary_POPdata_folder_path + '/urbanzones/'
    createFolder(outfolder)
    new_df.to_file(outfolder+ '{0}_UrbanZones.geojson'.format(i),driver='GeoJSON',crs='EPSG:3035')
    #with open(path_rawPopData + '{}_summary.txt'.format(i),'w') as out:
        #df.info(buf = out)
#'COD_Z_URB', 'ZURB code'